In [1207]:
from pathlib import Path
import re
import fitz
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv
from typing import List
import faiss


In [1208]:
load_dotenv()
client = OpenAI()

In [1209]:
path = Path("/Users/ozgur.sahin/Documents/ragchat_local/docs/domain1/2024_1689.pdf")

In [1210]:
doc = fitz.open(path)
page_sentences = {
    "sentence" : [],
    "is_header" : [],
    "page_num" : [],
    "boost" : []
 } 
for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        block_text = page.get_text("blocks")
        blocks = page.get_text("dict")["blocks"]
        text_blocks = [block for block in blocks if block["type"] == 0]
        for i,block in enumerate(text_blocks):
            if "lines" in block and len(block["lines"]) >= 1 and len(block["lines"]) < 3:
                for line in block["lines"]:
                    for span in line["spans"]:
                        text = span["text"]
                        previous_text = text
                        if span["size"] > 8 and (span["font"].find("Medi") >0 or span["font"].find("Bold") >0 or span["font"].find("B") >0) and len(text) > 15 and text[0].isupper():
                            current_header = text
                            page_sentences["sentence"].append(text)
                            page_sentences["is_header"].append(1)
                            page_sentences["page_num"].append(page_num+1)
                            page_sentences["boost"].append(0)
                        elif len(text) > 15:
                            page_sentences["sentence"].append(text)
                            page_sentences["is_header"].append(0)
                            page_sentences["page_num"].append(page_num+1)
                            page_sentences["boost"].append(0)
            elif "lines" in block:
                for sent_num in range(len(block_text[i][4].split('. '))):
                        sentence = re.split(r'(?<=[.!?])\s+', block_text[i][4])[sent_num].strip()
                        sentence = re.sub(r'(\b\w+)\s*\n\s*(\w+\b)',r'\1 \2',sentence)
                        sentence = re.sub(r'(\w+)-\s+(\w+)', r'\1\2', sentence)
                        sentence = re.sub(r'[,()]\s*\n\s*(\w+)',r' \1',sentence)
                        sentence = re.sub(r'(\b\w+)\s*-\s*(\w+\b)',r'\1 \2',sentence)
                        sentence = re.sub(r'(\w+)\s*[-–]\s*(\w+)',r'\1\2',sentence)
                        sentence = sentence.replace('\n','')
                        if len(sentence) > 15:
                            page_sentences["sentence"].append(sentence)
                            page_sentences["is_header"].append(0)
                            page_sentences["page_num"].append(page_num + 1)
                            page_sentences["boost"].append(0)

In [1211]:
print(page_sentences["sentence"])
print(page_sentences["boost"])
print(page_sentences["is_header"])

['REGULATION (EU) 2024/1689 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL', 'laying down harmonised rules on artificial intelligence and amending Regulations (EC) No 300/2008, (EU) No 167/2013, (EU) No 168/2013, (EU) 2018/858, (EU) 2018/1139 and (EU) 2019/2144 and Directives 2014/90/EU, (EU) 2016/797 and (EU) 2020/1828 (Artificial Intelligence Act)', '(Text with EEA relevance)', 'THE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE EUROPEAN UNION,', 'Having regard to the Treaty on the Functioning of the European Union, and in particular Articles 16 and 114 thereof,', 'Having regard to the proposal from the European Commission,', 'After transmission of the draft legislative act to the national parliaments,', 'Having regard to the opinion of the European Economic and Social Committee (', 'Having regard to the opinion of the European Central Bank (', 'Having regard to the opinion of the Committee of the Regions (', 'Acting in accordance with the ordinary legislative procedure (', '(1 The purpos

In [1214]:
headers_list = {
    "header" : [],
    "sentence_index" : [],
    "distance" : [],    
    "boost_value" : [],
}
for i,(sentence,is_header) in enumerate(zip(page_sentences["sentence"],page_sentences["is_header"])):
    if is_header == 1:
        headers_list["header"].append(sentence)
        headers_list["sentence_index"].append(i)

print(headers_list["header"])
print(headers_list["sentence_index"])

['REGULATION (EU) 2024/1689 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL', 'GENERAL PROVISIONS', 'PROHIBITED AI PRACTICES', 'Prohibited AI practices', 'HIGH-RISK AI SYSTEMS', 'Classification of AI systems as high-risk', 'Classification rules for high-risk AI systems', 'Amendments to Annex III', 'Requirements for high-risk AI systems', 'Compliance with the requirements', 'Risk management system', 'Data and data governance', 'Technical documentation', 'Transparency and provision of information to deployers', 'Accuracy, robustness and cybersecurity', 'Obligations of providers and deployers of high-risk AI systems and other parties', 'Obligations of providers of high-risk AI systems', 'Quality management system', 'Documentation keeping', 'Automatically generated logs', 'Corrective actions and duty of information', 'Cooperation with competent authorities', 'Authorised representatives of providers of high-risk AI systems', 'Obligations of importers', 'Obligations of distributors', 'Responsi

In [1215]:
def create_vector_embeddings_from_sentences(
            sentences: List[str],
            batch_size: int = 2000
        ):
        file_embeddings = []
        batches = [sentences[i:i+batch_size] for i in range(0,len(sentences), batch_size)]
        
        for batch in batches:
            sentence_embedding = client.embeddings.create(
                model="text-embedding-ada-002", input=batch
            )
            file_embeddings.extend(sentence_embedding.data)

        return np.array(
            [x.embedding for x in file_embeddings], float
        )

In [1216]:
def create_vector_embedding_from_query(query):
        query_embedding = client.embeddings.create(
            model="text-embedding-ada-002", input=query
        )
        return np.array(query_embedding.data[0].embedding, float).reshape(1, -1)

In [1217]:
def create_flat_index(embeddings:np.ndarray
        ):
        dimension = len(embeddings[0])
        index = faiss.IndexFlatL2(dimension)
        index.add(embeddings)
        return index

In [1218]:
header_embed = create_vector_embeddings_from_sentences(headers_list['header'])

In [1219]:
query = "As a high-risk AI system provider, what are the information I should give to my users?"
query_embed = create_vector_embedding_from_query(query)

In [1220]:
index = create_flat_index(header_embed)

In [1221]:
header_dist = {
    "header" : [],
    "distance" : [],
}
D,I = index.search(query_embed,5)
for i,sentence_num in enumerate(I[0]):
    header_dist["header"].append(headers_list['header'][sentence_num])
    header_dist["distance"].append(D[0][i])
header_dist

{'header': ['Obligations of providers of high-risk AI systems',
  'Requirements for high-risk AI systems',
  'Information to be submitted upon the registration of high-risk AI systems in accordance with ',
  'Obligations of providers and deployers of high-risk AI systems and other parties',
  'Classification of AI systems as high-risk'],
 'distance': [0.22459358, 0.22863974, 0.24536313, 0.24783172, 0.25372368]}

In [1222]:
header_boost = {
    "header" : [],
    "boost" : []
}
header_boost["header"] = header_dist['header']
header_boost["boost"] =  [0.5,0.4,0.3,0.2,0.1]
header_boost

{'header': ['Obligations of providers of high-risk AI systems',
  'Requirements for high-risk AI systems',
  'Information to be submitted upon the registration of high-risk AI systems in accordance with ',
  'Obligations of providers and deployers of high-risk AI systems and other parties',
  'Classification of AI systems as high-risk'],
 'boost': [0.5, 0.4, 0.3, 0.2, 0.1]}

In [1226]:
for header in header_boost['header']:
    if header in headers_list['header']:
        sentence_index = headers_list['header'].index(header)
        start = headers_list['sentence_index'][sentence_index]
        if sentence_index+1 < len(page_sentences['sentence']):
            end = headers_list['sentence_index'][sentence_index+1] 
        else: 
            end = len(page_sentences['boost'])
    for i in range(start,end):
        page_sentences['boost'][i] = 1

In [1227]:
a = page_sentences["sentence"].index('Information to be submitted upon the registration of high-risk AI systems in accordance with ')
print(page_sentences["sentence"][a:a+100])
print(page_sentences["boost"][a:a+100])
print(page_sentences["page_num"][a:a+100])

['Information to be submitted upon the registration of high-risk AI systems in accordance with ', 'Section A — Information to be submitted by providers of high-risk AI systems in accordance with Article 49(1)', 'The following information shall be provided and thereafter kept up to date with regard to high-risk AI systems to be ', 'registered in accordance with Article 49(1):', 'The name, address and contact details of the provider;', 'The name, address and contact details of the authorised representative, where applicable;', 'A basic and concise description of the information used by the system (data, inputs) and its operating logic;', 'The status of the AI system (on the market, or in service; no longer placed on the market/in service, recalled);', 'A scanned copy of the certificate referred to in point 8, where applicable;', 'A copy of the EU declaration of conformity referred to in Article 47;', 'A URL for additional information (optional).', 'Section B — Information to be submitted